In [2]:
# !pip install -q unsloth==0.6.5 accelerate>=0.30.1 bitsandbytes datasets peft transformers==4.41.1
!pip install -q unsloth==0.6.5 'accelerate>=0.30.1' bitsandbytes datasets peft transformers==4.41.1
import os
from datasets import load_dataset
from unsloth import FastLanguageModel, Trainer, TrainingArguments, SFTTrainer, get_peft_model
from transformers import AutoTokenizer

ERROR: Could not find a version that satisfies the requirement unsloth==0.6.5 (from versions: 2024.8, 2024.9, 2024.9.post1, 2024.9.post2, 2024.9.post3, 2024.9.post4, 2024.10.0, 2024.10.1, 2024.10.2, 2024.10.4, 2024.10.5, 2024.10.6, 2024.10.7, 2024.11.2, 2024.11.4, 2024.11.5, 2024.11.6, 2024.11.7, 2024.11.8, 2024.11.9, 2024.11.10, 2024.11.11, 2024.12.1, 2024.12.2, 2024.12.3, 2024.12.4, 2024.12.5, 2024.12.6, 2024.12.7, 2024.12.8, 2024.12.9, 2024.12.10, 2024.12.11, 2024.12.12, 2025.1.1, 2025.1.2, 2025.1.3, 2025.1.4, 2025.1.5, 2025.1.6, 2025.1.8, 2025.2.2, 2025.2.3, 2025.2.4, 2025.2.5, 2025.2.6, 2025.2.7, 2025.2.8, 2025.2.9, 2025.2.10, 2025.2.11, 2025.2.12, 2025.2.13, 2025.2.14, 2025.2.15, 2025.3.1, 2025.3.2, 2025.3.3, 2025.3.4, 2025.3.5, 2025.3.6, 2025.3.7, 2025.3.8, 2025.3.9, 2025.3.10, 2025.3.11, 2025.3.12, 2025.3.13, 2025.3.14, 2025.3.15, 2025.3.16, 2025.3.17, 2025.3.18, 2025.3.19, 2025.4.1, 2025.4.2, 2025.4.3, 2025.4.4, 2025.4.5, 2025.4.7, 2025.5.1, 2025.5.2, 2025.5.3, 2025.5.4, 2025.

ModuleNotFoundError: No module named 'datasets'

In [ ]:
MODEL_NAME = "Qwen/Qwen1.5-7B-Chat"
OUTPUT_DIR = "outputs/qwen-finetuned"
DATA_PATH = "path/to/your_dataset.jsonl"  # update to actual file
TEXT_FIELD = "text"                       # update if column name differs
MAX_SEQ_LENGTH = 2048
BATCH_SIZE = 1
GRADIENT_ACCUMULATION = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3
WARMUP_RATIO = 0.03
SAVE_STEPS = 500
torch_dtype = "bfloat16"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
dataset = load_dataset("json", data_files=DATA_PATH)
train_dataset = dataset["train"]

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=torch_dtype,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    warmup_ratio=WARMUP_RATIO,
    fp16=False,
    bf16=torch_dtype == "bfloat16",
    logging_steps=10,
    save_steps=SAVE_STEPS,
    evaluation_strategy="no",
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    report_to="none",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    dataset_text_field=TEXT_FIELD,
    max_seq_length=MAX_SEQ_LENGTH,
    args=training_args,
    packing=True,
)

trainer.train()


In [ ]:
model = FastLanguageModel.merge_lora(model, tokenizer)
model.save_pretrained(OUTPUT_DIR, safe_serialization=False)
tokenizer.save_pretrained(OUTPUT_DIR)

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=OUTPUT_DIR, tokenizer=OUTPUT_DIR, max_new_tokens=256)
prompt = "### Instruction:\nExplain the benefits of knowledge graphs.\n\n### Response:"
print(pipe(prompt)[0]["generated_text"])